In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain-core langchain_openai python-dotenv langsmith pydantic spotipy

In [2]:
%pip install --quiet -U jupyterlab-lsp
%pip install --quiet -U "python-lsp-server[all]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
## Setup logging
import logging
import os
from dotenv import load_dotenv

load_dotenv(override=True)
logger = logging.getLogger(__name__)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',  # Define the format
    handlers=[logging.StreamHandler()]  # Output to the console
)

# Define Search Tool

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults(
    max_results=50,
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # search_depth="advanced",
    # include_domains = []
    # exclude_domains = []
)
name = search_tool.get_name()
desc = search_tool.description
desc

'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.'

# Create ToolNode

In [5]:
from langgraph.prebuilt import ToolNode
from spotify_tools import get_playlists, create_spotify_playlist, get_track_list_from_playlist, add_tracks_to_playlist, filter_artists, get_artists_from_playlist, find_similar_artists, find_top_tracks
from plan import validate_plan

tools = [get_playlists, create_spotify_playlist, add_tracks_to_playlist, filter_artists, validate_plan, get_artists_from_playlist, search_tool, find_similar_artists, get_track_list_from_playlist, find_top_tracks]
# tools = [get_playlists, get_track_list, create_spotify_playlist, add_tracks_to_playlist, search_tool]
tool_node = ToolNode(tools)

# Bind Tools to Model

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=os.getenv("OPENAI_MODEL_NAME"), temperature=1.0)
# llm_with_tools = llm.bind_tools(tools, strict=True, parallel_tool_calls=False)
llm_with_tools = llm.bind_tools(tools, strict=True)

# System Prompt

In [7]:
from langchain_core.messages import SystemMessage
from prompts import Prompts

system_message = SystemMessage(Prompts.SYSTEM)


# First Message

In [8]:
from langchain_core.messages import HumanMessage
from prompts import Prompts

human_message = HumanMessage(Prompts.SPOTIFY)
ai_message = llm_with_tools.invoke([system_message, human_message])

2024-11-14 09:15:47,801 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [9]:
if ai_message.tool_calls is None or ai_message.tool_calls[0]["name"] != "validate_plan":
    raise ValueError("should be tool message calling 'validate_plan'")

In [10]:
ai_message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  validate_plan (call_99aaGewOyDzPciZz48L6Mkg3)
 Call ID: call_99aaGewOyDzPciZz48L6Mkg3
  Args:
    plan: {'steps': [{'step_number': 1, 'name': 'get-new-rock-and-blues-artists', 'type': 'action', 'description': "Retrieve the list of unique artists from the 'New Rock and Blues' Spotify playlist.", 'success_criteria': 'List of unique artists is retrieved.', 'tool': 'functions.get_artists_from_playlist', 'action': "Use the tool to get the list of artists from the 'New Rock and Blues' playlist.", 'condition': None, 'substeps': None}, {'step_number': 2, 'name': 'find-similar-artists-loops', 'type': 'loop', 'description': "Find similar artists for each artist obtained from the 'New Rock and Blues' playlist.", 'success_criteria': 'Similar artists found for each artist in the list.', 'tool': 'functions.find_similar_artists', 'action': None, 'condition': 'For each artist in the list.', 'substeps': [{'st

In [11]:
plan = ai_message.tool_calls[0]["args"]
plan

{'plan': {'steps': [{'step_number': 1,
    'name': 'get-new-rock-and-blues-artists',
    'type': 'action',
    'description': "Retrieve the list of unique artists from the 'New Rock and Blues' Spotify playlist.",
    'success_criteria': 'List of unique artists is retrieved.',
    'tool': 'functions.get_artists_from_playlist',
    'action': "Use the tool to get the list of artists from the 'New Rock and Blues' playlist.",
    'condition': None,
    'substeps': None},
   {'step_number': 2,
    'name': 'find-similar-artists-loops',
    'type': 'loop',
    'description': "Find similar artists for each artist obtained from the 'New Rock and Blues' playlist.",
    'success_criteria': 'Similar artists found for each artist in the list.',
    'tool': 'functions.find_similar_artists',
    'action': None,
    'condition': 'For each artist in the list.',
    'substeps': [{'step_number': 1,
      'name': 'find-similar-artists',
      'type': 'action',
      'description': 'Use the tool to find sim

# chat_prompt_template holds all messages (Human, AI, Tool)

In [12]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt_template: ChatPromptTemplate = system_message + human_message + ai_message
messages = chat_prompt_template.format_messages()
messages

[SystemMessage(content="\n    You are an AI language assistant designed to execute tasks with precision and attention to detail. Your primary objective is to follow the user's instructions thoroughly and exactly as specified, leaving no detail unaddressed.\n\n    Guidelines:\n\n    - **Carefully Read Instructions:** Before starting any task, read all user instructions thoroughly to ensure complete understanding.\n\n    - **Create a Detailed Plan:** Develop a step-by-step plan that addresses every aspect of the user's requirements. Validate this plan before execution.\n\n    - **Utilize Advanced Planning Structures:** Do not limit yourself to linear or sequential plans. Employ loops for repetitive tasks, branches for conditional logic, and other programming constructs to handle complex tasks and decision-making processes.\n\n    - **Follow Instructions Precisely:** Execute each step exactly as described, without omitting or altering any part of the instructions.\n\n    - **Be Thorough:*

# Check for Tools in Message

In [13]:
from langchain_core.messages import ToolMessage

tools_in_ai_message = []
tools_by_name = {tool.name: tool for tool in tools}
messages = chat_prompt_template.format_messages()
for tool_call in messages[-1].tool_calls:
    tool = tools_by_name[tool_call["name"]]
    tools_in_ai_message.append(tool)
    # observation = tool.invoke(tool_call["args"])
    # result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
if len(tools_in_ai_message) == 0:
    raise ValueError
else:
    print(tools_in_ai_message)

[StructuredTool(name='validate_plan', description='Validates a step-by-step plan to solve a problem\n\nArgs:\n    plan (Plan): a step-by-step plan to solve a problem\n\nReturns:\n    bool: whether plan is okay or not', args_schema=<class 'langchain_core.utils.pydantic.validate_plan'>, func=<function validate_plan at 0x000001ECC497B7E0>)]


# Tool Call (should be validate_plan())

In [14]:
response = tool_node.invoke({"messages": messages})
tool_message = response["messages"][0]
tool_message.pretty_print()

================================= Tool Message =================================
Name: validate_plan

true


# Add Tool Response to Messages

In [15]:
chat_prompt_template += tool_message
chat_prompt_template.format_messages()

[SystemMessage(content="\n    You are an AI language assistant designed to execute tasks with precision and attention to detail. Your primary objective is to follow the user's instructions thoroughly and exactly as specified, leaving no detail unaddressed.\n\n    Guidelines:\n\n    - **Carefully Read Instructions:** Before starting any task, read all user instructions thoroughly to ensure complete understanding.\n\n    - **Create a Detailed Plan:** Develop a step-by-step plan that addresses every aspect of the user's requirements. Validate this plan before execution.\n\n    - **Utilize Advanced Planning Structures:** Do not limit yourself to linear or sequential plans. Employ loops for repetitive tasks, branches for conditional logic, and other programming constructs to handle complex tasks and decision-making processes.\n\n    - **Follow Instructions Precisely:** Execute each step exactly as described, without omitting or altering any part of the instructions.\n\n    - **Be Thorough:*

# Send Messages with Tool result to LLM

In [16]:
ai_tool_call_message = llm_with_tools.invoke(chat_prompt_template.format_messages())
ai_tool_call_message.pretty_print()

2024-11-14 09:15:49,921 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  get_playlists (call_pO2s8vIWm7nbNGmVC0flkk86)
 Call ID: call_pO2s8vIWm7nbNGmVC0flkk86
  Args:


# Validate correct tool call

In [17]:
if ai_tool_call_message.tool_calls is None or ai_tool_call_message.tool_calls[0]["name"] != "get_playlists":
    raise ValueError("should be tool message calling 'validate_plan'")

# Tool Call (should be get_playlists())

In [18]:
chat_prompt_template += ai_tool_call_message
messages = chat_prompt_template.format_messages()
response = tool_node.invoke({"messages": messages})

In [19]:
from langchain_core.messages import ToolMessage

playlist_tool_message: ToolMessage = response["messages"][0]
playlist_tool_message.pretty_print()

================================= Tool Message =================================
Name: get_playlists

{"playlists": [{"uri": "spotify:playlist:2dbYK5b7J0F7IdH5n1TEUK", "name": "RPreacher"}, {"uri": "spotify:playlist:4ack9YtUhdxRayJDAqlfQe", "name": "RP Bossa Nova Chill "}, {"uri": "spotify:playlist:75NW18NgdZuZeifrcjxKlZ", "name": "GVF"}, {"uri": "spotify:playlist:5iyONtUO21O88xw8pBblwh", "name": "Now And Then"}, {"uri": "spotify:playlist:4SDSUMg2HJcGFmHlupsU7U", "name": "Jazz classics"}, {"uri": "spotify:playlist:5fBxLG2wLAPVpx83402rHl", "name": "Acdc"}, {"uri": "spotify:playlist:17Yxm5hQbkbZwDcCTuiDXC", "name": "einaudi"}, {"uri": "spotify:playlist:2X6GccWfIqPWO6VYx405nO", "name": "Eric Clapton Blues"}, {"uri": "spotify:playlist:3smVXe8y8w8nt4lFH0HBkW", "name": "Alchemy: Dire Straits"}, {"uri": "spotify:playlist:2ifRZBXnX7UmIPUlGet5WZ", "name": "Blues"}, {"uri": "spotify:playlist:5nn2ePtd9gYEEDL3XnACCB", "name": "brazil"}, {"uri": "spotify:playlist:6YAntoKNXuogPKdV26b6XE", "name": "C

# Add Tool Response to Messages

In [20]:
chat_prompt_template += playlist_tool_message
messages = chat_prompt_template.format_messages()

# Send Playlists Results to LLM

In [21]:
ai_tool_call_message = llm_with_tools.invoke(messages)
ai_tool_call_message.pretty_print()

2024-11-14 09:18:36,401 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  get_artists_from_playlist (call_1EP3UmWCzx4r04E3IcAVORJa)
 Call ID: call_1EP3UmWCzx4r04E3IcAVORJa
  Args:
    playlist_id: spotify:playlist:01tk0aitEuGK0ajWCkzdKc


# Validate correct tool call

In [22]:
if ai_tool_call_message.tool_calls is None or ai_tool_call_message.tool_calls[0]["name"] != "get_artists_from_playlist":
    raise ValueError("should be tool message calling 'get_artists_from_playlist'")

# Add AI Tool Call to Messages

In [23]:
chat_prompt_template += ai_tool_call_message
messages = chat_prompt_template.format_messages()

# Tool Call (get_artists_from_playlist())

In [24]:
response = tool_node.invoke({"messages": messages})
track_list_tool_message: ToolMessage = response["messages"][0]

# Decode AI Tool Call message

In [25]:
track_list_tool_message.pretty_print()

================================= Tool Message =================================
Name: get_artists_from_playlist

{"artists": {"Gary Clark Jr.": "spotify:artist:01aC2ikO4Xgb2LUpf9JfKp", "Ayron Jones": "spotify:artist:1iEaqWaYpKo9x0OrEq7Q7z", "Thirty Seconds To Mars": "spotify:artist:0RqtSIYZmd4fiBKVFqyIqD", "KALEO": "spotify:artist:7jdFEYD2LTYjfwxOdlVjmc", "Arctic Monkeys": "spotify:artist:7Ln80lUS6He07XvHI8qqHH", "Queens of the Stone Age": "spotify:artist:4pejUc4iciQfgdX6OKulQn", "Black Pistol Fire": "spotify:artist:0Nrwy16xCPXG8AwkMbcVvo", "Welshly Arms": "spotify:artist:1xKrH6GSh9CJh8nYwbqW7B", "The Record Company": "spotify:artist:6vYg01ZFt1nREsUDMDPUYX", "The Heavy": "spotify:artist:0bZCak2tcRMY1dzEIuwF42", "Greta Van Fleet": "spotify:artist:4NpFxQe2UvRCAjto3JqlSl", "Whiskey Myers": "spotify:artist:26opZSJcXshCmCwxgZQmBc", "Colter Wall": "spotify:artist:3xYXYzm9H3RzyQgBrYwIcx", "Tyler Childers": "spotify:artist:13ZEDW6vyBF12HYcZRr4EV", "Chris Stapleton": "spotify:artist:4YLtscXsxb

In [ ]:
# json.loads(track_list_tool_message.content)

# Add Tool Response to Messages

In [26]:
chat_prompt_template += track_list_tool_message
messages = chat_prompt_template.format_messages()
messages

[SystemMessage(content="\n    You are an AI language assistant designed to execute tasks with precision and attention to detail. Your primary objective is to follow the user's instructions thoroughly and exactly as specified, leaving no detail unaddressed.\n\n    Guidelines:\n\n    - **Carefully Read Instructions:** Before starting any task, read all user instructions thoroughly to ensure complete understanding.\n\n    - **Create a Detailed Plan:** Develop a step-by-step plan that addresses every aspect of the user's requirements. Validate this plan before execution.\n\n    - **Utilize Advanced Planning Structures:** Do not limit yourself to linear or sequential plans. Employ loops for repetitive tasks, branches for conditional logic, and other programming constructs to handle complex tasks and decision-making processes.\n\n    - **Follow Instructions Precisely:** Execute each step exactly as described, without omitting or altering any part of the instructions.\n\n    - **Be Thorough:*

# Send Tool Result to LLM

In [27]:
response = llm_with_tools.invoke(messages)

2024-11-14 09:20:06,002 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# AI Tool Call (find_similar_artist())

In [ ]:
if ai_tool_call_message.tool_calls is None or ai_tool_call_message.tool_calls[0]["name"] != "get_artists_from_playlist":
    raise ValueError("should be tool message calling 'validate_plan'")

In [28]:
response.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  find_similar_artists (call_GV7H9t6hvoZNu8TNqger5Nbg)
 Call ID: call_GV7H9t6hvoZNu8TNqger5Nbg
  Args:
    artists: ['spotify:artist:01aC2ikO4Xgb2LUpf9JfKp', 'spotify:artist:1iEaqWaYpKo9x0OrEq7Q7z', 'spotify:artist:0RqtSIYZmd4fiBKVFqyIqD', 'spotify:artist:7jdFEYD2LTYjfwxOdlVjmc', 'spotify:artist:7Ln80lUS6He07XvHI8qqHH', 'spotify:artist:4pejUc4iciQfgdX6OKulQn', 'spotify:artist:0Nrwy16xCPXG8AwkMbcVvo']
  find_similar_artists (call_BynT7v5QQmUiLqtHe8UZpKyY)
 Call ID: call_BynT7v5QQmUiLqtHe8UZpKyY
  Args:
    artists: ['spotify:artist:1xKrH6GSh9CJh8nYwbqW7B', 'spotify:artist:6vYg01ZFt1nREsUDMDPUYX', 'spotify:artist:0bZCak2tcRMY1dzEIuwF42', 'spotify:artist:4NpFxQe2UvRCAjto3JqlSl', 'spotify:artist:26opZSJcXshCmCwxgZQmBc', 'spotify:artist:3xYXYzm9H3RzyQgBrYwIcx', 'spotify:artist:13ZEDW6vyBF12HYcZRr4EV']
  find_similar_artists (call_xEIyEPEIpptTRAI5i0vo8j1x)
 Call ID: call_xEIyEPEIpptTRAI5i0vo8j1x
  Ar

In [ ]:
if ai_tool_call_message.tool_calls is None or ai_tool_call_message.tool_calls[0]["name"] != "find_similar_artists":
    raise ValueError("should be tool message calling 'validate_plan'")

# Add AI Tool Call to Messages

In [ ]:
chat_prompt_template += response
messages = chat_prompt_template.format_messages()
messages

# Tool Call (find_similar_artists())

In [ ]:
response = tool_node.invoke({"messages": messages})

In [ ]:
tool_message = response["messages"][0]
tool_message.pretty_print()

# Add Tool Response to Messages

In [ ]:
chat_prompt_template += tool_message
messages = chat_prompt_template.format_messages()
messages

# Send Tool Result to LLM

In [ ]:
response = llm_with_tools.invoke(messages)
response.pretty_print()

In [ ]:
chat_prompt_template += response
messages = chat_prompt_template.format_messages()
messages

In [ ]:
hm = HumanMessage("Why did you only asked for similar artists for a ubset of all artists in the playlist? How can the prompt be improved to make sure you go through all of them?")
chat_prompt_template += hm
response = llm_with_tools.invoke(messages)
response.pretty_print()

In [ ]:
filtered_artists: ToolMessage = response["messages"][0]
filtered_artists.pretty_print()

In [ ]:
response = llm_with_tools.invoke(messages)

In [ ]:
chat_prompt_template += response
chat_prompt_template.format_messages()

In [ ]:
response = tool_node.invoke({"messages": [response]})

In [ ]:
response